# Projeto 1 - Ciência dos Dados

Nome: João Otávio Palma

Nome: Pedro Garcia Carneiro da Cunha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
## Carregando algumas bibliotecas:

In [80]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## Classificador automático sobre a Bolsa de valores brasileira (B3)


As notícias tiveram o seu escopo dentro da parte de economia do IstoéDinheiro. Com isso, o foco o target foi a bolsa brasileira, avaliando as informações nacionais e internacionais que poderiam causar um aumento da bolsa ou uma queda da mesma. Dessa forma, houve uma separação entre assuntos irrelevantes(pouco impactantes ou com nenhum realcionamento com o target), neutros(indiferença na manipulação da bolsa de valores brasileira) e relevantes(possuem chance de gerar alguma alteração). 

## "Bloco" de funções que auxiliaram o classificador automático:

### Cleanup --> retira caracteres especiais e os caracteres que encontram-se em Caps lock:

In [81]:
def cleanup(text):

    #importando a string
    punctuation = '[\n!-:.?;,")(~''/|\*&%@#$_><}{ºª+=`]' 
    
    pattern = re.compile(punctuation)
    
    text_subbed = re.sub(pattern, '', text)
    
    return text_subbed.lower()
## Teste
###textinho = 'hoje eu e o pedrão fomos ~ao shopping? talvez sim, talvez nao! \n mas quem sabe - algumas pessoas foram: magali, cebolinha, magazine luiza (guilda)'

###txtinho = cleanup(textinho)
###print(txtinho)

### NLP (Natural Language Processing) --> uso de bibliotecas que tenham stopwords para melhorar o processamento do calssificador:

In [82]:
def stpwd (l_p): # recebe uma lista de palavras para analisar
    
    l_p_filtradas = []
    
    p_stpwd = stopwords.words('portuguese') # Acessa a lista de "stop words" da língua portuguesa dentro da biblioteca NLTK
    
    for p in l_p: # p = palarvra
        
        if p not in p_stpwd: # verifica se a palavra não está presente na lista
            
            l_p_filtradas.append(p)
    
    return l_p_filtradas

## Verificando onde o arquivo se encontra:

In [83]:
print('Esperamos trabalhar no diretório:')
print(os.getcwd())

Esperamos trabalhar no diretório:
C:\Users\pedro\Documents\GitHub\22-2b-cd-p1-grupo_joaoogp\notebooks


## Carregando a base de dados com as notícias classificadas manualmente:

In [84]:
os.listdir('data')

['dados.xlsx']

In [85]:
filename = 'data/dados.xlsx'

# Verificando se é possível achar o arquivo:

if 'dados.xlsx' in os.listdir('data'):
    print(f'Parece que o arquivo {filename} está na pasta diretório correta')
    
else:
    print(f'Parece que o arquivo {filename} NÃO está na pasta diretório correta')
    

Parece que o arquivo data/dados.xlsx está na pasta diretório correta


In [86]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,"Investimento Direto no País soma US$ 4,483 bi ...",Os Investimentos Diretos no País (IDP) somaram...,26/08/2022 10:23,47,2
1,economia,‘Não queremos a chinesada entrando aqui quebra...,"O ministro da Economia, Paulo Guedes, afirmou ...",26/08/2022 17:08,46,0
2,economia,Presidente de petroleira russa morre ao cair d...,"MOSCOU (Reuters) – Ravil Maganov, presidente d...",01/09/2022 16:51,22,1
3,economia,"PPI da Alemanha avança 37,2% em julho na compa...","O índice de preços ao produtor (PPI, na sigla ...",19/08/2022 07:19,72,1
4,economia,Problema em traje de cosmonauta russo força fi...,Por Joey Roulette WASHINGTON (Reuters) – Um do...,17/08/2022 16:47,78,1


In [87]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,Bolsonaro cita 1964 e ano do impeachment de Di...,Antes de sair do Palácio da Alvorada nesta qua...,07/09/2022 09:40,8,1
1,economia,Investidores não veem guinada do Fed e se prep...,Por Gertrude Chavez-Dreyfuss NOVA YORK (Reuter...,24/08/2022 08:10,57,1
2,economia,Ministro diz que enviou estudo do Porto de San...,"O ministro da Infraestrutura, Marcelo Sampaio,...",01/09/2022 13:06,24,0
3,economia,Missão da AIEA na usina de Zaporizhzhia “pode ...,KIEV (Reuters) – A missão de monitoramento da ...,01/09/2022 08:30,26,1
4,economia,Bancos admitem ‘licença’ de R$ 70 bilhões,Representantes de bancos e fundos de investime...,16/08/2022 17:00,82,2


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Tratamento de base de treinamento: Limpando o texto e separação da base com base no Target

In [88]:
#Limpando a base Train

train.fillna("",inplace=True)
train['Descrição'] = train['Descrição'].apply(cleanup)
train['Titulo'] = train['Titulo'].apply(cleanup)

#Separando as noticias pela target

not_positiva = train.loc[train['Target'] == 'Positiva']
not_neutra = train.loc[train['Target'] == 'Neutra']
not_negativa = train.loc[train['Target'] == 'Negativa']


Classificando de acordo com o título da notícia:

In [89]:
#Criando uma lista com os títulos de acordo com o target
title_positiva = list(not_positiva['Titulo'])
title_neutra = list(not_neutra['Titulo'])
title_negativa = list(not_negativa['Titulo'])


#Notícias divididas em palavras e sem stopwords
palavras_title_positiva = stpwd(' '.join(title_positiva).split())
palavras_title_neutra = stpwd(' '.join(title_neutra).split())
palavras_title_negativa = stpwd(' '.join(title_negativa).split())


#Criando um conjunto que contém todas as palavras que estão presentes no título
palavras_port_titulo = titulo_positiva_palavras + titulo_negativa_palavras + titulo_neutra_palavras
palavras_port_titulo = list(dict.fromkeys(portugues_titulo))

title_positiva_series = pd.Series(palavras_title_positiva)
title_neutra_series = pd.Series(palavras_title_neutra)
title_negativa_series = pd.Series(palavras_title_negativa)


#Descobrindo a fequencia absoluta que as palavras aparecem nas notícias
title_positiva_abs = title_positiva_series.value_counts()
title_neutras_abs = title_neutra_series.value_counts()
title_negativa_abs = title_negativa_series.value_counts()

NameError: name 'titulo_positiva_palavras' is not defined

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**